In [1]:
path_data = '/data'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
unique_topics = data_unfiltered['topics'].unique()

In [7]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [8]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [9]:
len(data)

36754

### Split topics

In [10]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [11]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [12]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [13]:
topics = dict_topic2img.keys()

# Create prompts for the text encoder

In [14]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [15]:
#English_topics

In [16]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [17]:
topic2english['family snapshots']

'This is a photo of family snapshots'

In [18]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


### Load img embeddings

In [21]:
import pickle
with open("nllb_img_embedding.pkl", 'rb') as f:
    img_embedding = pickle.load(f)

In [22]:
img_embedding.shape

torch.Size([36754, 1152])

## Cosine sim for default english_prompts and imgs

In [25]:
from sentence_transformers import SentenceTransformer, util
from open_clip import get_tokenizer

from open_clip import create_model_from_pretrained
from PIL import Image, ImageFile
import timm

In [26]:
# Load model directly
from transformers import AutoProcessor, AutoModel

model, transform = create_model_from_pretrained("nllb-clip-large-siglip", "v1", device="cuda")
model = model.eval()

In [27]:
from open_clip import get_tokenizer

tokenizer = get_tokenizer("nllb-clip-large-siglip")

In [29]:
#list_english_score_dicts = []
for english_prompt in tqdm.tqdm([english_dict_topic2imgGT]):
    text = list(english_prompt.keys())
    text_langs = 'eng_Latn'
    text_inputs = []
    for i in range(len(text)):
        tokenizer.set_language(text_langs)
        text_inputs.append(tokenizer(text[i]))

    text_inputs = torch.stack(text_inputs).squeeze(1)
    text_inputs = text_inputs.to("cuda")
    
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    
    #text_emb = text_model.encode(text, convert_to_tensor=True, show_progress_bar=False)
    cossim = util.cos_sim(text_features, img_embedding)
    cos_sim = cossim.cpu().numpy()
    
    dict_topic2score = {}
    for i, topic in enumerate(english_prompt.keys()):
        clip_scores = cos_sim[i] # clip scores between topic and images from data['imageRelPath']
        
        list_scores_img = list(zip(clip_scores, data['imageRelPath'], data['topics'])) # ******
        dict_topic2score[topic] = sorted(list_scores_img, key = lambda x: x[0], reverse=True)
        
    #list_english_score_dicts.append(dict_topic2score)

100%|██████████| 1/1 [00:09<00:00,  9.52s/it]


In [30]:
len(dict_topic2score)

270

In [31]:
import pickle
f = open("dict_topic2score.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(dict_topic2score,f)

# close file
f.close()

In [13]:
import pickle
with open("dict_topic2score.pkl", 'rb') as f:
    dict_topic2score = pickle.load(f)

### Score Processing

In [33]:
labels=["poor", "lower-middle", "upper-middle", "rich"]

#list_of_topic2prompt_dict[1]
data["quartile"] = pd.qcut(data["income"], q=[0, 0.25, 0.5, 0.75, 1],    
                         labels=labels)

In [34]:
# map each image to corresponding topics, countries, income level ... TODO
dict_img2topic, dict_img2country, dict_img2incomelevel, dict_country2region = {}, {}, {}, {}
for list_topics, image_path, country, incomelevel, region in zip(data['topics'], data['imageRelPath'], data['country.name'], data['quartile'], data['region.id']):
    dict_img2topic[image_path] = list_topics
    dict_img2country[image_path] = country
    dict_img2incomelevel[image_path] = incomelevel
    dict_country2region[country] = region

print(len(dict_img2topic))
# print(dict_img2country)

36754


In [35]:
for topic in dict_topic2score.keys():
    print(topic)

This is a photo of family snapshots
This is a photo of cutlery
This is a photo of family
This is a photo of place where eating dinner
This is a photo of plate of food
This is a photo of table with food
This is a photo of get water
This is a photo of stove/hob
This is a photo of light source in kitchen
This is a photo of drinking water
This is a photo of kitchen sink
This is a photo of bathroom privacy
This is a photo of backyard
This is a photo of front door
This is a photo of home
This is a photo of source of cool
This is a photo of street view
This is a photo of teeth
This is a photo of hand back
This is a photo of hand palm
This is a photo of pet
This is a photo of bedroom
This is a photo of grains
This is a photo of wardrobe
This is a photo of armchair
This is a photo of hallway
This is a photo of kitchen
This is a photo of books
This is a photo of everyday shoes
This is a photo of hand washing
This is a photo of trash/waste
This is a photo of tools
This is a photo of hair brush/co

In [36]:
## Trial for the first country_dict  (recall and precision and F1 scores are equal)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_images = list(data['imageRelPath'])
list_dict_topic2FNimg = []
list_dict_topic2F1 = []

#for country_dict in tqdm.tqdm(list_english_score_dicts):
    
dict_topic2FNimg = {}
dict_topic2F1 = {}

for topic in tqdm.tqdm(dict_topic2score.keys()):
    dict_country_topic2img = english_dict_topic2imgGT
    count_pos = len(dict_country_topic2img[topic])

    GT_pos_images = dict_country_topic2img[topic]
    Pred_pos_images = [data[1] for data in dict_topic2score[topic][:count_pos]]

    GT_labels = [1 if img in GT_pos_images else 0 for img in all_images]
    Pred_labels = [1 if img in Pred_pos_images else 0 for img in all_images]


    accuracy = round(accuracy_score(GT_labels, Pred_labels) * 100, 2)
    precision = round(precision_score(GT_labels, Pred_labels) * 100, 2)
    recall = round(recall_score(GT_labels, Pred_labels) * 100, 2)
    f1 = round(f1_score(GT_labels, Pred_labels) * 100, 2)

    print(f"Acc: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}")

    dict_topic2F1[topic] = f1 
    list_dict_topic2F1.append(dict_topic2F1)


    # What are the forgotten items (FN) 
    images_FN = []
    for idx, img in enumerate(all_images):
        label_GT = GT_labels[idx]
        label_Pred = Pred_labels[idx]
        #FN
        if label_GT == 1 and label_Pred == 0:
            images_FN.append(img) 

    dict_topic2FNimg[topic] = images_FN

list_dict_topic2FNimg.append(dict_topic2FNimg)


  0%|          | 1/270 [00:00<00:54,  4.98it/s]

Acc: 99.57, Recall: 22.55, Precision: 22.55, F1: 22.55


  1%|          | 2/270 [00:00<01:10,  3.79it/s]

Acc: 98.71, Recall: 21.26, Precision: 21.26, F1: 21.26


  1%|          | 3/270 [00:00<01:30,  2.95it/s]

...

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0


 99%|█████████▉| 268/270 [01:03<00:00,  6.85it/s]

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0


100%|██████████| 270/270 [01:03<00:00,  4.23it/s]

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0


In [37]:
len(dict_topic2FNimg)

270

In [38]:
len(dict_topic2FNimg['This is a photo of family snapshots'])

79

In [69]:
len(list_dict_topic2F1[0])

272

In [71]:
list_dict_topic2F1[6]

{'This is a photo of family snapshots from Myanmar': 5.88,
 'This is a photo of cutlery from Myanmar': 26.91,
 'This is a photo of family from Myanmar': 33.98,
 'This is a photo of place where eating dinner from Myanmar': 11.64,
 'This is a photo of plate of food from Myanmar': 26.26,
 'This is a photo of table with food from Myanmar': 38.3,
 'This is a photo of get water from Myanmar': 7.33,
 'This is a photo of stove/hob from Myanmar': 31.0,
 'This is a photo of light source in kitchen from Myanmar': 17.48,
 'This is a photo of drinking water from Myanmar': 12.73,
 'This is a photo of kitchen sink from Myanmar': 26.46,
 'This is a photo of bathroom privacy from Myanmar': 12.74,
 'This is a photo of backyard from Myanmar': 13.36,
 'This is a photo of front door from Myanmar': 41.31,
 'This is a photo of home from Myanmar': 13.28,
 'This is a photo of source of cool from Myanmar': 8.64,
 'This is a photo of street view from Myanmar': 27.08,
 'This is a photo of teeth from Myanmar': 57.

In [39]:
f = open("data/english_dict_topic2FNimg.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(dict_topic2FNimg,f)

# close file
f.close()

In [40]:
f = open("data/english_dict_topic2F1.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(dict_topic2F1,f)

# close file
f.close()